# Thermodynamic Relations

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 
from mmf_hfb import tf_completion as tf
from mmf_hfb.FuldeFerrelState import FFState
import itertools
from mmfutils.plot import imcontourf
tf.MAX_DIVISION = 200
clear_output()

## 1D Example(Not Ground State)

In [ ]:
def case_1d():
    plt.figure(figsize(10,5))
    mu = 5
    dmu = 3
    delta = 2
    k_c = 200
    dim = 1
    q=1
    dqs = np.linspace(0., 3, 40)*q
    ff = FFState(mu=mu, dmu=dmu, delta=delta, dim=dim, k_c=k_c,fix_g=True)
    states =[ff.check_superfluidity(mu=mu, dmu=dmu, q=q, dq=dq) for dq in dqs]
    ps =[ff.get_pressure(mu=mu, dmu=dmu, q=q, dq=dq).n for dq in dqs]
    plt.subplot(211)
    plt.plot(dqs, states,'-', label='States')
    plt.legend()
    plt.title(f'$\mu=${mu},d$\mu$={dmu}, $\Delta$={delta}, q={q}', fontsize=16)
    plt.subplot(212)
    plt.plot(dqs, ps, 'o', label='SF Pressure')
    ps =[ff.get_pressure(mu=mu, dmu=dmu, q=q, delta=0, dq=dq).n for dq in dqs] # normal states
    plt.plot(dqs, ps,'--', label='NS Pressure')
    plt.axvline(q)  
    plt.xlabel(f'$\delta q$', fontsize=16)
    plt.legend()
    na, nb = ff.get_densities(mu=mu, dmu=dmu, q=q, dq=1.4)
    return (na.n, nb.n)                              

In [ ]:
na,nb=case_1d()

In [ ]:
na, nb

In [ ]:
dq=1.5
na * (1+ dq) + nb * (1 - dq)

In [ ]:
def case_1d_spectrum(mu=5, dmu=3, delta=2, q=1, L=0, R=6, qN=10, dqN=40):
    k_c = 200
    dim = 1
    qs= np.linspace(0,1,qN) * q
    dqs = np.linspace(L, R, dqN) 
  
    ff = FFState(mu=mu, dmu=dmu, delta=delta, dim=dim, k_c=k_c,fix_g=True)
    
    def get_state(q, dq):
        dq = dq * q
        return ff.check_superfluidity(mu=mu, dmu=dmu, q=q, dq=dq)
    def get_press(q, dq, d=None):
        dq = dq * q
        return ff.get_pressure(mu=mu, dmu=dmu, delta=d, q=q, dq=dq).n
    def reshape(x):
        return np.array(x).reshape(qN, dqN)
    ss = []
    ps = []
    pn = []
    for q in qs:
        for dq in dqs:
            ss.append(get_state(q, dq))
            ps.append(get_press(q, dq))
            pn.append(get_press(q, dq, d=0))
                      
    return (qs, dqs, reshape(ss), reshape(ps), reshape(pn))

In [ ]:
qs, dqs, ss, ps, pn = case_1d_spectrum(mu=5, dmu=1.55, delta=2, q=1, N=8, qN=10, dqN=30)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
imcontourf(dqs, qs, ss.T)
plt.subplot(132)
imcontourf(dqs, qs, ps.T)
plt.colorbar()
plt.subplot(133)
imcontourf(dqs, qs, pn.T)
plt.colorbar()

In [ ]:
ret1 = case_1d_spectrum(mu=5, dmu=1.95, delta=2, q=1, N=8, qN=10, dqN=30)

In [ ]:
def plots(ret):
    qs, dqs, ss, ps, pn  = ret
    plt.figure(figsize=(15,5))
    plt.subplot(131)
    imcontourf(dqs, qs, ss.T)
    plt.subplot(132)
    imcontourf(dqs, qs, ps.T)
    plt.colorbar()
    plt.subplot(133)
    imcontourf(dqs, qs, pn.T)
    plt.colorbar()

In [ ]:
plots(ret1)

In [ ]:
ret2 = case_1d_spectrum(mu=5, dmu=3, delta=2, q=1, L=-3, R=3, qN=10, dqN=30)

In [ ]:
plots(ret2)

In [ ]:
ret3 = case_1d_spectrum(mu=5, dmu=3, delta=2, q=1.2, N=8, qN=10, dqN=30)

\begin{align}
k_a &= k + q + dq, \qquad k_b = k+q - dq ,\qquad
\epsilon_a = \frac{\hbar^2}{2m}k_a^2 - \mu_a, \qquad \epsilon_b = \frac{\hbar^2}{2m}k_b^2 - \mu_b,\\
E&=\sqrt{\epsilon_+^2+\abs{\Delta}^2},\qquad \omega_+= \epsilon_-+E, \qquad \omega_- = \epsilon_- - E\\
\epsilon_+&= \frac{\hbar^2}{4m}(k_a^2+k_b^2) - \mu_+= \frac{\hbar^2}{2m}\left[(k+q)^2 + dq^2\right] - \mu_+\\
\epsilon_-&= \frac{\hbar^2}{4m}(k_a^2-k_b^2) - \mu_-=\frac{\hbar^2}{m}(k +q)dq - \mu_-\tag{1}\\
\end{align}

# Analytical Results for Free Fermi Gas
* Check numerical results against analytical derivaions,[ECE 407 Farhan Rana]
[ECE 407 Farhan Rana]: https://courses.cit.cornell.edu/mse5470/handout3.pdf 'Farhan Rana "Free Electron Gas in 2D and 1D"'

### Energy Density for 1D free Fermi Gas($\Delta=0$)
$$
\mathcal{E}=\frac{\sqrt{2m}}{\pi\hbar}\frac{E_F^{3/2}}{3}\qquad n=\frac{\sqrt{2mE_F}}{\pi \hbar}\tag{2}
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{\sqrt{2m}}{\pi\hbar}\frac{\mu_a^{3/2}+\mu_b^{3/2}}{3}=\frac{\sqrt{2m}}{\pi\hbar}\frac{(\mu+d\mu)^{3/2}+(\mu-d\mu)^{3/2}}{3}
\end{align}

### Energy Density for 2D free Fermi Gas($\Delta=0$)
For single component
$$
\mathcal{E}=\frac{m}{4\pi\hbar^2}E_F^2,\qquad n=\frac{m}{2\pi\hbar^2}E_F, \qquad \mathcal{E}=\frac{1}{2}nE_F\tag{3}
$$

Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{m}{4\pi\hbar^2}(\mu_a^2+\mu_b^2)=\frac{m}{4\pi\hbar^2}\left((\mu+d\mu)^2+(\mu-d\mu)^2\right)\\
\end{align}

### Energy Density for 3D free Fermi Gas($\Delta=0$)
For single component
$$
\mathcal{E}=\frac{\hbar^2}{10m\pi^2}k_F^5=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}E_F^{5/2}, \qquad n=\frac{k_F^3}{3\pi^2}=\frac{1}{3\pi^2}\frac{(2m)^{3/2}}{\hbar^3}E_F^{3/2}\tag{4}
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}(\mu_a^{5/2}+\mu_b^{5/2})=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}\left((\mu+d\mu)^{5/2}+(\mu+d\mu)^{5/2}\right)\\
\end{align}

\begin{align}
\frac{d E}{d n}
&=\frac{d E(n_a, n_b)}{d n}\\
&=\frac{\partial E(n_a, n_b)}{\partial n_a}\frac{\partial n_a}{\partial n}+\frac{\partial E(n_a, n_b)}{\partial n_b}\frac{\partial n_b}{\partial n}\\
&=\frac{1}{2}\left[\frac{\partial E(n_a, n_b)}{\partial n_a}+\frac{\partial E(n_a, n_b)}{\partial n_b}\right]\\
&=\frac{\mu_a + \mu_b}{2}=\mu\tag{6}
\end{align}
* Since $n=n_+=n_a + n_b$, $n_-=n_a - n_b$, so $n_a = (n_++n_-)/2$, $n_b = (n_+-n_-)/2$, then $\frac{\partial n_a}{\partial n_+}=1/2$

# Conditions for FF State
* Thermodymamcally, for given $\mu$,$d\mu$, the pressure should be maximized with respect to $q$ and $dq$
* $dq>q$, and dq can be many times of q, which means 2q is the speed difference
* $dq=\frac{n_+}{n_-}$, smaller density difference means bigger dq, or small q will give better chance to find a FF State

In [ ]:
mu = 5
dmu = 1
delta = 0.5
k_c = 200
dim = 1
if dim == 3:
    k_c = 50

In [ ]:
ff = FFState(mu=mu, dmu=dmu, delta=delta, dim=dim, k_c=k_c,fix_g=True)
ff.bSuperfluidity

In [ ]:
dmus = np.linspace(0.7,1.6,15)
bSuperfluidities = [ff.check_superfluidity(mu=mu, dmu=dmu, q=0, dq=0) for dmu in  dmus]
plt.plot(dmus, bSuperfluidities)
if dim==1:
    min_dmu = 1.0
    max_dmu= 1.15
elif dim == 2:
    min_dmu=2.7
    max_dmu=4.1
plt.axvline(min_dmu)
plt.axvline(max_dmu)
ff.check_superfluidity(mu=mu, dmu=max_dmu, q=0, dq=0)

In [ ]:
dmu=.95
ff.get_densities(mu=mu, dmu=dmu), ff.check_superfluidity(mu=mu, dmu=dmu)

In [ ]:
ff.isFFS(mu=mu, dmu=dmu)

In [ ]:
dmus = np.linspace(min_dmu, max_dmu, 10)
dmu_qs = []

In [ ]:
qs = np.linspace(0, 5, 100)
for dmu in dmus:
    max_q = 0
    for q in reversed(qs):
        if ff.check_superfluidity(mu=mu, dmu=dmu, q=q * dmu, dq=0):
            max_q = q * dmu
            break
    print(dmu, max_q)
    dmu_qs.append((dmu, max_q))

In [ ]:
ff.check_superfluidity(mu=mu, dmu=2.91222222222222222, q=.7, dq=.01)

In [ ]:
id=0
plt.figure(figsize=(20,15))
for dmu_q in dmu_qs:
    id=id+1
    plt.subplot(3,4,id)
    dmu, q_max = dmu_q
    q = q_max * 1
    dqs = np.linspace(0, 2, 20)*q
    flags =[ff.check_superfluidity(mu=mu, dmu=dmu, q=q, dq=dq) for dq in dqs]
    ps =[ff.get_pressure(mu=mu, dmu=3, q=q, dq=dq).n for dq in dqs]
    plt.plot(dqs, flags)
    plt.plot(dqs, ps)
    plt.axvline(q)
    plt.title(f"$d\mu$={dmu:4.4},q={q:4.4}")

In [ ]:
ff.bSuperfluidity,ff.bStateSentinel

# Search for FF States

In [ ]:
ff.bStateSentinel=True
ff.bPolorized = True
for dmu_q in reversed(dmu_qs):
    dmu, max_q = dmu_q
    print(dmu_q)
    qs = np.linspace(0.001, max_q, 20)
    dqs = np.linspace(1.001, 2, 10)
    for q in qs:
        print(f"dmu={dmu},q={q}")
        ps = []
        for dq_ in dqs:
            try:
                dq = dq_ * q
                press = ff.get_pressure(mu=mu, dmu=dmu, q=q, dq=dq)
                ps.append(press.n)
            except:
                # print(f"dmu={dmu},q={q},dq={dq}")
                # if len(ps) > 1 and ps[1]>ps[0]:
                #       print(q,dq, dmu, ps)
                continue
        print(ps)
        for i in range(1, len(ps) - 1):
            if ps[i]>= ps[i-1]:
                print(f"FF State: dmu={dmu}, q={q}, dq={dq}")
                break

# A specific point

In [ ]:
dmu=1.3777777777777778
q=6.888888888888889
dq=0.8888888888888888
dqs = np.linspace(0, 2 * dq, 10) 
ps = []
for dq in dqs:
    try:
        ps.append(ff.get_pressure(mu=mu, dmu=dmu, q=q, dq=dq).n)
    except:
        break

plt.plot(dqs[:len(ps)], ps)